<a href="https://colab.research.google.com/github/hack-mans/PanoHead/blob/main/PanoHead_colab_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 00 - Check GPU status (requires 8GB VRAM)



In [ ]:
!nvidia-smi

# 01 - Setup

Clone the PanoHead repo

In [ ]:
%cd /content
!git clone https://github.com/hack-mans/PanoHead.git
%cd PanoHead

Install requirements

In [ ]:
!pip install imgui glfw pyspng mrcfile ninja plyfile trimesh

Fetch the models

In [ ]:
!mkdir '/content/PanoHead/models'
!gdown 1FqvQzICV1H4fbQaz8BiWxtiRYxJd4T8N -O '/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl'
!gdown 1bYg9jCM9KJB2qxhFpktUDL_UwsaaitSS -O '/content/PanoHead/models/baseline-easy-khair-025000.pkl'
!gdown 1Qb4vD7Ylbt9BLHVqa2uljvdy0jNRB1hd -O '/content/PanoHead/models/ablation-trigridD-1-025000.pkl'

# 02 - Setup Datasets (unused currently - WIP)

Upload your images to /content/datasets/PanoHead/datasets/custom_img

In [ ]:
!mkdir /content/PanoHead/datasets/custom_img
import os
os.chdir('content/PanoHead/datasets/custom_img')
from google.colab import files
uploaded - files.upload()

Upload your segmentation masks to /content/PanoHead/datasets/custom_seg

In [ ]:
!mkdir /content/PanoHead/datasets/custom_seg
import os
os.chdir('content/PanoHead/datasets/custom_seg')
from google.colab import files
uploaded - files.upload()

# 03A - Generating Results (optional)

Generate videos using pre-trained model

In [ ]:
!python gen_videos.py --network models/easy-khair-180-gpc0.8-trans10-025000.pkl --seeds 0-3 --grid 2x2 --outdir=out --cfg Head --trunc 0.7

# 03B - Applications

Generate full head reconstruction 360 videos and PLY 3D mesh from single RGB image
(using easy-khair-180-gpc0.8-trans10-025000.pkl)

In [ ]:
%cd /content/PanoHead
# Perform the pti and save w
!python projector_withseg.py --outdir="/content/PanoHead/pti_out" --target_img="/content/PanoHead/dataset/testdata_img" --target_seg="/content/PanoHead/dataset/testdata_seg" --network "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl" --idx 0
# Generate .mp4 before finetune
!python gen_videos_proj_withseg.py --output="/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/pre.mp4" --latent="/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz" --trunc 0.7 --network "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl" --cfg Head
# Generate .mp4, .ply mesh and frame images after finetune
!python gen_videos_proj_withseg.py --output="/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/post.mp4" --latent="/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz" --trunc 0.7 --network "/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl" --cfg Head --shapes True --frames True --level 42

# 04 Display Results

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
video_path = '/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/post.mp4'

# Compressed video path
compressed_path = "/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/post_compressed.mp4"

os.system(f"ffmpeg -i {video_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=512 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# 05 Download Results

In [ ]:
import shutil
from google.colab import files

def zip_folder(folder_path, output_path):
  shutil.make_archive(output_path, 'zip', folder_path)

# Zip the results folder
zip_folder('/content/PanoHead/pti_out/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render', '/content/panohead_results')

# Download the zip file
files.download('/content/panohead_results.zip')